# Rooflines 
> Plotting rooflines with Altair

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]

In [ ]:
#hide
import numpy as np
import pandas as pd

import altair as alt
import csv

In [ ]:
#hide
## Just some needed functions
#Function to read from a csv file and return a numpy 2D array
def read_from_csv(filename):  
    array= []
    with open(filename, newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            array.append(row)
    array = np.asarray(array)
    return array

In [ ]:
#hide
## Just some needed functions
def load_and_display(filenames):
    dataframes=[]
    for filename in filenames:
        table = read_from_csv(filename)  # To read from a csv file into a 2D numpy array
        dataframe = pd.DataFrame(data=table[2:,:], columns=[table[0,0:], table[1,0:]])  #To transform to dataframe the first and second row will be header
        dataframe.loc[dataframe.duplicated(dataframe.columns[0]) , dataframe.columns[0]] = ''  #To remove duplicates from first column
        dataframes.append(dataframe)     #To save all dataframes in here
    return dataframes

# Overview of the experiments done

In [ ]:
#hide_input
filenames = ['Data/imagenet.csv', 'Data/mnist.csv', 'Data/cifar10.csv']
dataframes = load_and_display(filenames)
for i in range(len(dataframes)):    
    display(HTML(dataframes[i].to_html(index=False)))

# Rooflines for all hardware platforms with Neural Networks arithmetic Intensity

In [1]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
#load dataset and plot it
rooflines(pd.read_csv("data/processed_csv/rooflines_hardware_neural_networks.csv"))

alt.LayerChart(...)

# Performance Predictions for IMAGENET, MNIST and CIFAR

In [ ]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
#load dataset and plot it
heatmap(pd.read_csv("data/processed_csv/performance_prediction_imagenet_mnist_cifar10.csv"), 
        'red', 
        'Performance Predictions for IMAGENET, MNIST and CIFAR10')